In [3]:
pip install pulp cplex scipy

  Using cached pulp-3.2.1-py3-none-any.whl.metadata (6.9 kB)
  Using cached cplex-22.1.2.0-cp311-cp311-manylinux2014_x86_64.whl.metadata (56 kB)
Using cached pulp-3.2.1-py3-none-any.whl (16.4 MB)
Using cached cplex-22.1.2.0-cp311-cp311-manylinux2014_x86_64.whl (44.3 MB)


The general form of LP problems we will consider is:

        min_x  cᵀx
   subject to  Gx ≥ h
               Ax = b
               l ≤ x ≤ u

The benchmark datasets use the .mps format for LP problems so there is a function to extract the parameters from this file type into the form above. But since I haven't figured out how to access these datasets yet there is also a function to generate large feasible LP example problems in the .mps format. Finally, there is a preliminary function that implements the PDHG algorithm but not yet any of the enhancments.

# Initialization

In [65]:
#Comment out numpy for cupy generation
#import numpy as np
import cupy as cp
from scipy.sparse import random as sparse_random
import pulp

# This function generates large feasible LP problems to test and saves them in the .mps format
def generate_feasible_lp(num_vars=100, num_ineq=200, num_eq=50, density=0.05, mps_filename="generated_lp.mps"):
    '''This function generates large feasible LP problems to test and saves them in the .mps format'''

    # The number of variables in each constraint with nonzero coefficients will be roughly density * num_vars

    rng = cp.random.default_rng(0)

    # Step 1: (Random) feasible solution
    x_feas = rng.uniform(low=-10, high=10, size=(num_vars, 1))

    # Step 2: Sparse matrices (convert to dense)
    G_sparse = sparse_random(num_ineq, num_vars, density=density, format='csr', random_state=None)
    A_sparse = sparse_random(num_eq, num_vars, density=density, format='csr', random_state=None)

    G = G_sparse.toarray()
    A = A_sparse.toarray()

    #Addition - convert to cupy array
    G = cp.array(G)
    A = cp.array(A)

    # Step 3: RHS vectors
    h = G @ x_feas + rng.uniform(0.1, 5.0, size=(num_ineq, 1))
    b = A @ x_feas

    # Step 4: Bounds
    l = x_feas - rng.uniform(1, 5, size=(num_vars, 1))
    u = x_feas + rng.uniform(1, 5, size=(num_vars, 1))
    l = cp.maximum(l, -1e4)
    u = cp.minimum(u, 1e4)

    # Step 5: Objective
    c = rng.normal(size=(num_vars, 1))

    # Step 6: Write to MPS using pulp
    prob = pulp.LpProblem("Feasible_LP", pulp.LpMinimize)
    x_vars = [
        pulp.LpVariable(f"x{i}", lowBound=float(l[i]), upBound=float(u[i]))
        for i in range(num_vars)
    ]
    prob += pulp.lpDot(c.flatten(), x_vars)

    # Inequality constraints: Gx ≥ h
    for i in range(num_ineq):
        prob += pulp.lpDot(G[i], x_vars) <= float(h[i]), f"ineq_{i}"

    # Equality constraints: Ax = b
    for i in range(num_eq):
        prob += pulp.lpDot(A[i], x_vars) == float(b[i]), f"eq_{i}"

    prob.writeMPS(mps_filename)
    print(f"LP written to: {mps_filename}")

In [66]:
#import numpy as np
import cplex
from cplex.exceptions import CplexError

# This function extracts the parameters of the LP from the .mps format to the general form
def mps_to_standard_form(mps_file):
    '''
    Extracts the parameters of the LP from the .mps format to the general form
    Returns c, G ineq matrix, h ineq vec, A eq matrix, b eq vec, l and u lower and
    upper box constraints
    '''
    try:
        cpx = cplex.Cplex(mps_file)
        cpx.set_results_stream(None)  # mute output

        # Number of variables and constraints
        num_vars = cpx.variables.get_num()
        num_constraints = cpx.linear_constraints.get_num()

        # Objective vector (c)
        c = cp.array(cpx.objective.get_linear())

        # Constraint matrix
        A_full = cpx.linear_constraints.get_rows()
        senses = cpx.linear_constraints.get_senses()
        rhs = cp.array(cpx.linear_constraints.get_rhs())

        A = []
        G = []
        b = []
        h = []

        for i, (row, sense, rhs_i) in enumerate(zip(A_full, senses, rhs)):
            row_vec = cp.zeros(num_vars)
            for idx, val in zip(row.ind, row.val):
                row_vec[idx] = val
            if sense == 'E':  # Equality constraint
                A.append(row_vec)
                b.append(rhs_i)
            elif sense == 'G':  # Greater than or equal
                G.append(row_vec)
                h.append(rhs_i)
            elif sense == 'L':  # Less than or equal
                # convert to -Gx ≥ -h
                G.append(-row_vec)
                h.append(-rhs_i)

        A = cp.array(A) if A else cp.zeros((0, num_vars))
        b = cp.array(b) if b else cp.zeros(0)
        G = cp.array(G) if G else cp.zeros((0, num_vars))
        h = cp.array(h) if h else cp.zeros(0)

        # Bounds
        l = cp.array(cpx.variables.get_lower_bounds())
        u = cp.array(cpx.variables.get_upper_bounds())

        c = c.reshape(-1, 1)
        h = h.reshape(-1, 1)
        b = b.reshape(-1, 1)
        l = l.reshape(-1, 1)
        u = u.reshape(-1, 1)

        return c, G, h, A, b, l, u

    except CplexError as e:
        print("CPLEX Error:", e)
        return None

In [67]:
import cupy as cp

def proj_Lam(lam):
    return lam

         # LP parameters, maximum iterations, error tolerance, period checks termination criteria
def pdhg(c, G, h, A, b, l, u, max_iter=1000, tol=1e-4, term_period=1000):
    """
    Solves:
        min cᵀx s.t. Gx ≥ h, Ax = b, l ≤ x ≤ u
    using PDHG algorithm.
    Returns minimal objective value, minimizer estimate in list format, and the number of iterations run
    """

    # Define Parameters
    K = cp.vstack([G, A])
    q = cp.vstack([h, b])

    eta = 0.9/cp.linalg.norm(K, 2)
    omega = 10

    tau = eta/omega
    sigma = eta*omega

    m_1 = G.shape[0]
    m_2 = A.shape[0]
    n = c.shape[0]

    # Termination Parameters
    tol_dual = tol * (1 + cp.linalg.norm(c))
    tol_prim = tol * (1 + cp.linalg.norm(q))

    # Initialize Primal and Dual Variables
    x = cp.minimum(cp.maximum(cp.zeros((n, 1)), l), u)
    y = cp.zeros((m_1 + m_2, 1))

    for i in range(1, max_iter + 1):
        # Primal update
        x_old = x.copy()
        # Project x onto box constraints l ≤ x ≤ u
        grad = c - K.T @ y
        x = cp.minimum(cp.maximum(x - tau * grad, l), u)

        # Dual update
        y += sigma * (q - K @ (2*x - x_old))
        y[:m_1] = cp.maximum(y[:m_1], 0)  # project onto constraint y:m_1 ≥ 0

        # Check Termination Criteria Periodically
        if i%term_period == 0:

            dual_op = (q.T @ y)[0][0]
            prim_op = (c.T @ x)[0][0]

            lam_p_op = (l.T @ cp.maximum(grad, 0))[0][0]
            lam_n_op = (u.T @ cp.minimum(grad, 0))[0][0]

            '''Print [uncomment] for debugging'''
            #print(dual_op + lam_p_op + lam_n_op, prim_op)
            #print(cp.linalg.norm(cp.vstack([A @ x - b, cp.maximum(h - G @ x, 0)])))
            print(cp.linalg.norm(cp.vstack([A @ x - b, cp.maximum(h - G @ x, 0)])), cp.linalg.norm(grad - proj_Lam(grad)), cp.abs(dual_op + lam_p_op - lam_n_op - prim_op))
            #print(tol_dual, tol_prim, tol * (1 + cp.abs(dual_op + lam_p_op - lam_n_op) + cp.abs(prim_op)))
            if (
                cp.linalg.norm(cp.vstack([A @ x - b, cp.maximum(h - G @ x, 0)])) <= tol_prim  # Primal Feasibility
                and cp.linalg.norm(grad - proj_Lam(grad)) <= tol_dual  # Dual Feasibility
                and cp.abs(dual_op + lam_p_op + lam_n_op - prim_op) <= tol * (1 + cp.abs(dual_op + lam_p_op + lam_n_op) + cp.abs(prim_op))  # Duality Gap
            ):
                break

    # Returns minimal objective value, minimizer estimate in list format, and the number of iterations run
    return (c.T @ x)[0][0], x.T[0].tolist(), i

# Testing the algorithm

In [68]:
from time import perf_counter

# Makes a timer to measure code omtimality
class Timer:
    def __enter__(self):
        self.start = perf_counter()
        return self
    def __exit__(self, *args):
        self.end = perf_counter()
        self.elapsed = self.end - self.start
        print(f"Elapsed time: {self.elapsed:.6f} seconds")


In [70]:
import cupy as cp
from scipy.sparse import random as sparse_random
import pulp
import numpy as np

# This function generates large feasible LP problems to test and saves them in the .mps format
def generate_feasible_lp(num_vars=100, num_ineq=200, num_eq=50, density=0.05, mps_filename="generated_lp.mps"):
    '''This function generates large feasible LP problems to test and saves them in the .mps format'''

    # The number of variables in each constraint with nonzero coefficients will be roughly density * num_vars

    rng = cp.random.default_rng(0)

    # Step 1: (Random) feasible solution
    x_feas = rng.uniform(low=-10, high=10, size=(num_vars, 1))

    # Step 2: Sparse matrices (convert to dense)
    G_sparse = sparse_random(num_ineq, num_vars, density=density, format='csr', random_state=None)
    A_sparse = sparse_random(num_eq, num_vars, density=density, format='csr', random_state=None)

    G = G_sparse.toarray()
    A = A_sparse.toarray()

    #cupy array
    G = cp.array(G)
    A = cp.array(A)

    # Step 3: RHS vectors
    h = G @ x_feas + rng.uniform(0.1, 5.0, size=(num_ineq, 1))
    b = A @ x_feas

    # Step 4: Bounds
    l = x_feas - rng.uniform(1, 5, size=(num_vars, 1))
    u = x_feas + rng.uniform(1, 5, size=(num_vars, 1))
    l = cp.maximum(l, -1e4)
    u = cp.minimum(u, 1e4)

    # Step 5: Objective
    c = cp.random.normal(size=(num_vars, 1))

    # Step 6: Write to MPS using pulp
    prob = pulp.LpProblem("Feasible_LP", pulp.LpMinimize)
    x_vars = [
        pulp.LpVariable(f"x{i}", lowBound=float(l[i].get()), upBound=float(u[i].get()))
        for i in range(num_vars)
    ]
    prob += pulp.lpDot(c.flatten().get(), x_vars)

    # Inequality constraints: Gx ≥ h
    for i in range(num_ineq):
        prob += pulp.lpDot(G[i].get(), x_vars) <= float(h[i].get()), f"ineq_{i}"

    # Equality constraints: Ax = b
    for i in range(num_eq):
        prob += pulp.lpDot(A[i].get(), x_vars) == float(b[i].get()), f"eq_{i}"

    prob.writeMPS(mps_filename)
    print(f"LP written to: {mps_filename}")

cplex version (for comparison)

In [71]:
import cplex

# Solve the LP using either primal simplex, dual simplex, or barrier (interior point).
# Only works for LP with no more than 1000 constraints and no more than 1000 variables
cpx = cplex.Cplex("large_example.mps")
c, G, h, A, b, l, u = mps_to_standard_form("large_example.mps")

# Times how long it takes to solve
with Timer():
    cpx.solve()

# Save the minimizer and minimal objective values for comparison
cpx_obj_val = cpx.solution.get_objective_value()
cpx_min = cpx.solution.get_values()
print("Objective value:", cpx_obj_val)
#print("Minimizer: xᵀ =", cpx_min)


Selected objective sense:  MINIMIZE
Selected objective  name:  OBJ
Selected RHS        name:  RHS
Selected bound      name:  BND

Selected objective sense:  MINIMIZE
Selected objective  name:  OBJ
Selected RHS        name:  RHS
Selected bound      name:  BND
Version identifier: 22.1.2.0 | 2024-12-10 | f4cec290b
CPXPARAM_Read_DataCheck                          1
Tried aggregator 1 time.
Linear dependency checker was stopped due to maximum work limit.
No LP presolve or aggregator reductions.
Presolve time = 0.05 sec. (88.64 ticks)

Iteration log . . .
Iteration:     1   Dual objective     =         -2351.676780
Iteration:    62   Dual objective     =         -1995.690468
Iteration:   124   Dual objective     =         -1921.555387
Iteration:   186   Dual objective     =         -1842.366068
Iteration:   248   Dual objective     =         -1784.710120
Iteration:   310   Dual objective     =         -1728.055604
Iteration:   372   Dual objective     =         -1682.150353
Iteration:   434

Our PDHG

In [72]:
# Extract LP parameters from generated example
with Timer():
    c, G, h, A, b, l, u = mps_to_standard_form("large_example.mps")
    pdhg_obj_val, pdhg_min, iterations = pdhg(c, G, h, A, b, l, u, max_iter=1000000, tol=1e-4)
print("Objective Value:", pdhg_obj_val)
print("Iterations:", iterations)
#print("Minimizer: xᵀ =",pdhg_min)

# The distance between the two minimizer solutions
# Should be small but won't be incredibly small since the vectors are high dimensional
#distance = cp.linalg.norm(cp.array(pdhg_min) - cp.array(cpx_min))
#print("Distance:", distance)


Selected objective sense:  MINIMIZE
Selected objective  name:  OBJ
Selected RHS        name:  RHS
Selected bound      name:  BND
0.8688290129274907 0.0 787.3007486389383
0.1868538872195678 0.0 801.0467273113345
0.11820840256056167 0.0 784.6623112192381
0.08894155392875758 0.0 749.9930547787772
0.058686442501278996 0.0 714.784626192622
0.04450081953254877 0.0 694.3810662526578
0.04011934975982188 0.0 756.1190196838978
0.03201496475974411 0.0 700.975470564488
0.028590343309746462 0.0 725.9857639753775
0.022016763381902924 0.0 710.1692455634127
0.01674598633624759 0.0 760.8010993792406
0.015964601165610236 0.0 708.9457591163577
0.01115776533571994 0.0 690.3273027292029
0.00855264936821312 0.0 706.7150676835178
0.0070803489883533985 0.0 697.2695161981705
0.00795383237959395 0.0 712.8734075020873
0.008208847091781773 0.0 679.8638413352389
0.006707775908358682 0.0 693.9760154728851
0.0062525194608344945 0.0 684.9623947971033
0.0070676400669694266 0.0 704.6731453759451
0.006539018645898381 0